In [ ]:
# can run this in your env if you dont have autotime
#!conda install -c conda-forge jupyterlab_execute_time
%load_ext autotime    

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# PART 1: AUTOGRAD

### What is a torch tensor?
 - alot like a numpy array
 - desinged for high performance
 - especially designed to calculate partical derivatives
 - the pytorch neural network module is meant to work with them

In [ ]:
# very simple start
X = torch.randn(5)

In [ ]:
X?

In [ ]:
X

In [ ]:
X.data

In [ ]:
X.grad

In [ ]:
X.dtype

In [ ]:
X.device

In [ ]:
T = 100
np_x = np.linspace(-3,3,T) 
np_t = np.linspace(0,2,T) 

tr_x = torch.linspace(-3,3,T, requires_grad=True)
tr_t = torch.linspace(0,2,T, requires_grad=True)

In [ ]:
def cube(x):
    return x**3 / 10

np_y = cube(np_x)
tr_y = cube(tr_x)



In [ ]:
tr_y.sum().backward()

### note: grad not stored in y, but x

In [ ]:
tr_y.grad

In [ ]:
fig, ax = plt.subplots(1,2)

#ax[0].plot(tr_x, tr_x.grad)
ax[0].plot(tr_x.data, tr_x.grad)

ax[1].plot(np_x[1:], np.diff(np_y)/(np_x[1]-np_x[0]))

titles=['AUTOGRAD','NUMERIC']
for a,title in zip(ax.ravel(),titles):
    a.set_title(title)

In [ ]:
#tr_x.grad = None

def multivar(x,t):
    assert type(x) == type(t), 'types must match'
    assert type(x) in [np.ndarray,torch.Tensor], 'type must be torch Tensor or np array'
    
    if type(x) == np.ndarray:
        return np.sin(3*x)/3 + np.log(t+.1)
    if type(x) == torch.Tensor:
        return torch.sin(3*tr_x)/3 + torch.log(tr_t+.1)
    
np_y = multivar(np_x, np_t)
tr_y = multivar(tr_x, tr_t)

## Calculating the partials is really easy with pytorch

In [ ]:
tr_y.sum().backward()

## less so using numerical derivative

### and, of course, the numerical one will have discretization errors

In [ ]:

# f(x+dx, t) - f(x, t)
dy_x = multivar(np_x[1:], np_t[1:]) - multivar(np_x[:-1], np_t[1:])

# f(x,t+dt) - f(x, t)
dy_t = multivar(np_x[:-1], np_t[1:]) - multivar(np_x[:-1], np_t[:-1])

dx = np_x[1]-np_x[0]
dt = np_t[1]-np_t[0]

## Let's plot our results...

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,8), sharey='col')
ax[0,0].plot(tr_x.detach(), tr_x.grad)
ax[0,1].plot(tr_t.detach(), tr_t.grad)

ax[1,0].plot(np_x[:-1], dy_x/dx)
ax[1,1].plot(np_t[:-1], dy_t/dt)

titles=['x.grad','t.grad','numeric x','numeric t']
for a,title in zip(ax.ravel(),titles):
    a.set_title(title)


# Part 2: nn.Module

In [ ]:
from torch import nn

### The crucial pieces of a neural network module are
 - #### forward method
 - #### nn.Parameters attribute

In [ ]:
class SimpleModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.init_params = torch.randn(2)
        self.weights = nn.Parameter( torch.tensor( [self.init_params[0],self.init_params[1]] ))
    def forward(self, input: torch.tensor) -> torch.tensor:
        return (self.weights * input**2).sum(dim=-1)

In [ ]:
circArea = SimpleModule()
test_point = torch.tensor([2,3])

In [ ]:
params = circArea.init_params;
print(params)

#compare the forward method
print( circArea(test_point) )

#to manual evaluation
print( params[0]*test_point[0]**2+params[1]*test_point[1]**2)

#### Let's say we are trying to learn the sum of the squares...

In [ ]:
input_data = torch.tensor(np.random.normal(0,1,(10,2)))
measured_data = (input_data**2).sum(dim=-1)

In [ ]:
circArea.train()

#L2 norm
loss = ( (circArea(input_data)-measured_data)**2).mean()
print(loss)

### just for clarity about what .backward does, lets take a look at the model parameters before and after
- ####  note None for gradients

In [ ]:
for p in circArea.parameters():
    print('params:', p,)
    print('gradients:',p.grad)

In [ ]:
loss.backward()

#### after .backward....

In [ ]:
for p in circArea.parameters():
    print('params:', p,)
    print('gradients:',p.grad)

In [ ]:
learning_rate = 1E-1
with torch.no_grad():
    for param in circArea.parameters():
        param -= learning_rate * param.grad
    

In [ ]:
for p in circArea.parameters():
    print('params:', p,)
    print('gradients:',p.grad)
    

## Note that .grad is no longer 'None'

### we will have to manually reset it

## Let's try it again!

In [ ]:
input_data = torch.tensor(np.random.normal(0,1,(10,2)), requires_grad=True)
measured_data = (input_data**2).sum(dim=-1)

loss = ( (circArea(input_data)-measured_data)**2).mean()

#DONT FORGET TO RESET THE GRAD
circArea.zero_grad()

#### calculate the gradient....

In [ ]:
loss.backward()

In [ ]:
for p in circArea.parameters():
    print('params:', p,)
    print('gradients:',p.grad)

In [ ]:
learning_rate = 1E-1
with torch.no_grad():
    for param in circArea.parameters():
        param -= learning_rate * param.grad
        
circArea.zero_grad()

for p in circArea.parameters():
    print(p, p.grad)

### Obviously, we should loop this...

In [ ]:
n_iter = 250
n_samples = 10
learning_rate = 1E-2

# we'll store results in these
losses = []
p_array = np.empty((n_iter,2))
grads = []


for i in range(n_iter):
    
    
    p_array[i,:] = circArea.weights.data
    
    input_data = torch.tensor(np.random.normal(0,1,(n_samples,2)), requires_grad=True)
    noise = torch.normal(0,.1,size=(n_samples,1))
    measured_data = ( input_data**2 + noise  ).sum(dim=-1)
    
    circArea.train()
    loss = ( (circArea(input_data)-measured_data)**2 ).mean()
    loss.backward()

    losses.append(loss.item())
            
    with torch.no_grad():
        for p in circArea.parameters():
            grads.append(p.grad)
            p -= learning_rate * p.grad 
        
    circArea.zero_grad()
    

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,4))


ax[0].plot(losses)

ax[1].plot(p_array)

ax[2].plot(grads)

titles=['LOSS','WEIGHTS','GRAD']
for a,title in zip(ax.ravel(),titles):
    a.set_title(title)

ax[0].set_yscale('log')


# PART 3: standard nn Layers and Optimizers

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleModule(nn.Module):
    def __init__(self):
        super().__init__()

        self.l_out = nn.Linear(5, 1)
                               
    def forward(self, input: torch.tensor) -> torch.tensor:
          return self.l_out(input)

#### what does this 'linear' layer do?

 - linear model of the form W $\cdot$ X + B

In [ ]:
myModel = SimpleModule()

test_point = torch.tensor([1,2,3,4,5]).float()

In [ ]:
for name, param in myModel.named_parameters():
    print(f'{name}:', param.data)

In [ ]:
w, b = [p.data for p in myModel.parameters()]

In [ ]:
print(w)
print(b)

In [ ]:
print('expected outcome value:',torch.matmul(w,test_point)+b)
print('model outcome:', myModel(test_point))

## OK, now let's learn something a small linear model like this can learn

In [ ]:

def target(data):    
    return 4*data[:,1] -  2*data[:,2] + data[:,3] - 3*data[:,4]+ 2

def get_data(N):
    data = torch.rand(size=(N,5))
    
    noise = torch.normal(0, .2, size=(1,N))
    
    labeled_data = target(data)+noise

    return data, labeled_data

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(12,3))
x, y = get_data(1000)

for i in range(5):
    ax[i].scatter(x[:,i], y, alpha=.1)

In [ ]:
fig, ax = plt.subplots()
with torch.no_grad():
    ax.scatter(y, myModel(x))
    ax.plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myModel(x)[:,0])**2).mean())

## Here we use a more standard pytorch training syntax
 - use a built in optimizer that does the learning algorithm
 - use model.train() and model.eval() when woking with pre-built layers
 - 


In [ ]:

optimizer = torch.optim.Adam(myModel.parameters(), lr=0.001)

n_iter = 1
#n_iter = 2000
n_samples = 500

losses =[]

verbose = True

def vprint(*args):
    if verbose:
        print(*args)

myModel.train()

for i in range(n_iter):
    d_in, d_out = get_data(n_samples)
    
    loss = ( (d_out-myModel(d_in)[:,0] )**2 ).mean()
    
    loss.backward()

    vprint('before optimizer')
    for name, param in myModel.named_parameters():
        vprint(f'{name}:', param.data)
        vprint('grad:'+f'{param.grad}')

    
    optimizer.step()

    vprint('after optimizer')
    for name, param in myModel.named_parameters():
        vprint(f'{name}:', param.data)
    
    losses.append(loss.item())

    myModel.zero_grad()
    
myModel.eval()

In [ ]:
fig, ax = plt.subplots(1,2)


with torch.no_grad():
    ax[0].plot(losses)
    ax[1].scatter(y, myModel(x))
    ax[1].plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myModel(x)[:,0])**2).mean())

In [ ]:
for name, param in myModel.named_parameters():
    print(f'{name}:', param.data)

## What if we have a more complicated function to learn?
 - ### still 5 inputs and 1 output, so maybe the same model will still work?

In [ ]:
myModel = SimpleModule()

def target(data):    
    return (torch.sign(data[:,0]-.5))* (torch.abs((torch.cos(6*data[:,1]) -  torch.exp( (data[:,2] - data[:,3])/3 ) - 3*data[:,4]**2 + 2 ) )) /2.5

def get_data(N):
    data = torch.rand(size=(N,5))
    
    noise = torch.normal(0, .1, size=(1,N))
    
    labeled_data = target(data) + noise

    return data, labeled_data


In [ ]:
fig, ax = plt.subplots(1,5, figsize=(12,3))
x, y = get_data(1000)

for i in range(5):
    ax[i].scatter(x[:,i], y, alpha=.1)

In [ ]:
fig, ax = plt.subplots()
with torch.no_grad():
    ax.scatter(y, myModel(x))
    ax.plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myModel(x)[:,0])**2).mean())

In [ ]:
optimizer = torch.optim.Adam(myModel.parameters(), lr=0.01)

n_iter = 1000
n_samples = 500

losses =[]

myModel.train()

for i in range(n_iter):
    d_in, d_out = get_data(n_samples)
    
    loss = ( (d_out-myModel(d_in)[:,0] )**2 ).mean()
    
    loss.backward()
            
    optimizer.step()

    losses.append(loss.item())

    myModel.zero_grad()
    
myModel.eval()

In [ ]:
fig, ax = plt.subplots(1,2)

with torch.no_grad():
    ax[0].plot(losses)
    ax[1].scatter(y, myModel(x))
    ax[1].plot(np.linspace(-2,2,10), np.linspace(-2,2,10))
    print( ((y-myModel(x)[:,0])**2).mean())

# Part 4: bigger nonlinear models, torch.device

 -  ## our simple model was not up to the task
 -  ## lets make a super overkill one!

In [ ]:

def get_homogenous_MLP(config):
    '''
    helper function to make fully connected MLP layers
    '''
    n_input = config.n_in
    n_output= config.n_out
    n_hidden = config.layer_size
    num_inner = config.n_inner
    
    linear_layers = [nn.Linear(n_input, n_hidden),
                    *[nn.Linear(n_hidden, n_hidden) for i in range(num_inner)],
                    nn.Linear(n_hidden, n_output),
                    ]
    nonlinear_layers = [nn.ReLU(inplace=True) for i in range(len(linear_layers)-1)]
    
    layers = [None]*(2*len(linear_layers)-1)
    
    layers[::2] = linear_layers
    layers[1::2] = nonlinear_layers
    
    return nn.Sequential(*layers)

# takes in a cofig to make an MLP
class SimpleMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.h = get_homogenous_MLP(self.config)
                               
    def forward(self, input: torch.tensor) -> torch.tensor:
          return self.h(input)


In [ ]:
from argparse import Namespace
options = Namespace()

options.n_in = 5
options.n_out = 1
options.n_inner = 2
options.layer_size = 128

myMLP = SimpleMLP(options)

In [ ]:
myMLP

In [ ]:
fig, ax = plt.subplots()
with torch.no_grad():
    ax.scatter(y, myMLP(x))
    ax.plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())

In [ ]:
optimizer = torch.optim.Adam(myMLP.parameters(), lr=0.001)

n_datasets = 5
n_epochs = 150
n_samples = 50_000

losses =[]

myMLP.train()

for i in range(n_datasets):
    d_in, d_out = get_data(n_samples)

    for j in range(n_epochs):
        myMLP.zero_grad()
            
        loss = ( (d_out-myMLP(d_in)[:,0] )**2 ).mean()
        loss.backward()
        optimizer.step()
    
        losses.append(loss.item()) 
myMLP.eval()

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(losses)

with torch.no_grad():
    ax[1].scatter(y, myMLP(x))
    ax[1].plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())

### That took like a whole minute!!! Who has that kind of time?!!?

 - #### now we get to the high performance part of pytorch

 note: prepare for some pre-planned errors in the following


In [ ]:
d_in.device

In [ ]:
myMLP = SimpleMLP(options)
optimizer = torch.optim.Adam(myMLP.parameters(), lr=0.001)

losses =[]

myMLP.train()

for i in range(n_datasets):
    d_in, d_out = get_data(n_samples)



    # SEND TO MPS
    d_in = d_in.to(torch.device('mps'))
    # SEND TO MPS

    
    for j in range(n_epochs):
        myMLP.zero_grad()
        
        loss = ( (d_out-myMLP(d_in)[:,0] )**2 ).mean()
        loss.backward()
        optimizer.step()
    
        losses.append(loss.item())
myMLP.eval()
    

### Take a second to see what the problem was...

In [ ]:
myMLP.to(torch.device('mps'))

### model.to(device) sends the model's parameters to the device;
 - #### model itself does not have a device attribute

In [ ]:
print(myMLP.device)

In [ ]:
for p in myMLP.parameters():
    print(p.device)

In [ ]:
myMLP = SimpleMLP(options)


# SEND TO MPS
myMLP.to(torch.device('mps'))
# SEND TO MPS


optimizer = torch.optim.Adam(myMLP.parameters(), lr=0.001)

losses =[]

myMLP.train()

for i in range(n_datasets):
    d_in, d_out = get_data(n_samples)

    d_in = d_in.to(torch.device('mps'))
    
    for j in range(n_epochs):
        myMLP.zero_grad()
            
        loss = ( (d_out-myMLP(d_in)[:,0] )**2 ).mean()
        loss.backward()
        optimizer.step()
    
        losses.append(loss.item())
myMLP.eval()

### Again, take a second to ID the problem

In [ ]:
myMLP = SimpleMLP(options)
myMLP.to(torch.device('mps'))

optimizer = torch.optim.Adam(myMLP.parameters(), lr=0.001)

losses =[]

myMLP.train()

for i in range(n_datasets):

    # SEND BOTH TO MPS
    d_in, d_out = [d.to(torch.device('mps')) for d in  get_data(n_samples) ]
    # SEND BOTH TO MPS

    
    for j in range(n_epochs):
        myMLP.zero_grad()
            
        loss = ( (d_out-myMLP(d_in)[:,0] )**2 ).mean()
        loss.backward()
        optimizer.step()
    
        losses.append(loss.item())   
myMLP.eval()

## NOW, all we have to do is copy over the plot function, right?

- #### right....?

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(losses)

with torch.no_grad():
    ax[1].scatter(y, myMLP(x))
    ax[1].plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())

### easy fix: just send x to the mps backend, right...?
 - #### right...?

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(losses)

with torch.no_grad():
    
    ax[1].scatter(y, myMLP(x.to(torch.device('mps'))))
    ax[1].plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())

## Takeaway: Keeping Track of Device is very necesarry, or you'll have workarounds along every corner
- ### That was not the last error we were gonna have to fix if we kept along that path...
- ### Simplest solution is a function, can also be done with a class or a decorator

In [ ]:
def train_mlp(model, get_data_function, training_device, num_datasets, num_epochs, num_samples):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    losses =[]
    
    model.to(training_device)
    model.train()

    for i in range(num_datasets):
        d_in, d_out = [ item.to(training_device) for item in get_data_function(num_samples)]
    
        for j in range(num_epochs):            
            model.zero_grad()
            
            loss = ( (d_out-myMLP(d_in)[:,0] )**2 ).mean()
            loss.backward()
            optimizer.step()
    
            losses.append(loss.item())
     
    model.to(torch.device('cpu'))
    model.eval()
    
    return losses

In [ ]:
myMLP = SimpleMLP(options)

fig, ax = plt.subplots(1)
with torch.no_grad():
    ax.scatter(y, myMLP(x))
    ax.plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())



In [ ]:
losses = train_mlp(myMLP, get_data, torch.device('mps'), n_datasets, n_epochs, n_samples)

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(losses)

with torch.no_grad():
    ax[1].scatter(y, myMLP(x))
    ax[1].plot([y.min(),y.max()], [y.min(),y.max()], c='k')
    print( ((y-myMLP(x)[:,0])**2).mean())